In [125]:
from gensim.models import FastText, Word2Vec
import pandas as pd
import numpy as np
from pymorphy2 import MorphAnalyzer
from scipy.spatial.distance import cosine

In [3]:
# !wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
# !wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

In [138]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

In [139]:
df

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive
...,...,...
111918,Но не каждый хочет что то исправлять:( http://...,negative
111919,скучаю так :-( только @taaannyaaa вправляет мо...,negative
111920,"Вот и в школу, в говно это идти уже надо(",negative
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",negative


In [140]:
df.iloc[0, 0]

'@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)'

In [141]:
morph = MorphAnalyzer()

In [142]:
def preprocess_text(text: str) -> str:
    return [morph.parse(i)[0].normal_form for i in text.split()]

In [143]:
df['tokens'] = df['text'].apply(preprocess_text)
sentences = df['tokens'].to_list()

In [151]:
df.reset_index(inplace=True)

In [108]:
word2vec = Word2Vec(sentences=sentences, size=200, window=5, min_count=5, workers=4)

In [113]:
def get_emb_vector(tokens):
    vector = np.zeros(200)
    for token in tokens:
        if token in word2vec.wv:
            vector += word2vec.wv[token]
            
    return vector

In [145]:
df['embedding'] = df['tokens'].apply(get_emb_vector)

In [146]:
df.head()

,text,label,tokens,embedding
0,"@first_timee хоть я и школота, но поверь, у на...",positive,"[@first_timee, хоть, я, и, школота,, но, повер...","[0.23598844930529594, -1.4317471431568265, 3.7..."
1,"Да, все-таки он немного похож на него. Но мой ...",positive,"[да,, всё-таки, он, немного, похожий, на, него...","[0.3188290596008301, 0.23747546784579754, 1.97..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive,"[rt, @katiacheh:, ну, ты, идиотка), я, испугат...","[-0.5563862472772598, -1.3325133873149753, 0.4..."
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive,"[rt, @digger2912:, ""кто, то, в, угол, сидеть, ...","[1.5630476921796799, -3.9679797978606075, 5.93..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive,"[@irina_dyshkant, вот, что, значит, страшилка,...","[-2.264138240367174, -1.660068929195404, 1.783..."


In [147]:
def get_cosine_similarity(v1, v2):
    return cosine(v1, v2)

In [167]:
def get_similar_tweets(tweet):
    tokens = preprocess_text(tweet)
    embedding = get_emb_vector(tokens)
    
    distances = df['embedding'].apply(lambda x: get_cosine_similarity(x, embedding))
    
    top5 = distances.sort_values(ascending=True)[:5]
    
    return top5.index

In [176]:
for idx in get_similar_tweets(df.loc[1, 'text']):
    print(df.loc[idx, 'text'] + '\n')

Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D

"Один дома"-фильм на все времена. конечно, ничего шеедврального, но она чертовски милый и добрый:)
#Home_Alone

@NessMayles он очаровательный))брюнет,не слишком высокий,с каштановыми волосами,не кудри,но все равно вьются хд.и у него ямочки на щеках

но бапы и шайни всё равно захватили моё сердце, ребята, соррьки((99((

Хоть иногда меня бесят мои одноклассники, но все таки они самые-самые лучшие:*



In [177]:
for idx in get_similar_tweets(df.loc[111920, 'text']):
    print(df.loc[idx, 'text'] + '\n')

@_music_lover_13 оооо, еще и Оля) какая прелесть)))

КАКОГО ХРЕНА МЕНЯ РЕТВИТЯТ? ЕЩЕ И НЕЗНАКОМЫЕ ЛЮДИ. о_О

@plo_otnik @scumbag_Judas какая еще телочка???????????????????????????????????????

А ещё распирает жуткий интерес, какую фотку выберет х)

@guselnikovaalen какой скоро? Ещё 3 урока:|



### Кажется, более-менее работает))